In [10]:
import IPython
from IPython.display import display, HTML

# Increase the IOPub data rate limit to 10 MB/s
display(HTML('''
<script>
function setNotebookOptions() {
  var notebook = Jupyter.notebook;
  var rate_limit = 1e7;  // Set the desired rate limit in bytes per second (e.g., 10 MB/s)
  notebook.kernel.execute("from IPython.core.interactiveshell import InteractiveShell\nInteractiveShell.ast_node_interactivity = 'all'\n");
  notebook.kernel.execute("import sys\nsys.stdout._flush = lambda: None\n");
  notebook.kernel.execute("import IPython\nIPython.get_ipython().kernel.do_one_iteration = function(){ return 0; }\n");
  notebook.kernel.execute("IPython.get_ipython().kernel._allow_stdin = true\n");
  notebook.kernel.execute("IPython.OutputArea.prototype._should_scroll = function(){return false;}\n");
  notebook.kernel.execute("var rate_limit = " + rate_limit + ";\n");
  notebook.kernel.execute("var comm_manager = IPython.notebook.kernel.comm_manager;\n");
  notebook.kernel.execute("var rate_limiter = function(comm, msg){\n"
                          "    var handler = function(msg){\n"
                          "        var raw_msg = msg.msg_content.content.data;\n"
                          "        var data = new DataView(raw_msg.buffer);\n"
                          "        var rate_start_time = data.getFloat64(0, true);\n"
                          "        var rate_end_time = data.getFloat64(8, true);\n"
                          "        var data_length = raw_msg.length - 16;\n"
                          "        var rate = (data_length / (rate_end_time - rate_start_time));\n"
                          "        if (rate > rate_limit){\n"
                          "            comm.kernel.execute('IPython.get_ipython().kernel.do_one_iteration = function(){ return 0; }');\n"
                          "            comm.kernel.execute('IPython.get_ipython().kernel._allow_stdin = true');\n"
                          "        }\n"
                          "        else{\n"
                          "            comm.kernel.execute('IPython.get_ipython().kernel.do_one_iteration = undefined;');\n"
                          "            comm.kernel.execute('IPython.get_ipython().kernel._allow_stdin = false');\n"
                          "        }\n"
                          "    }\n"
                          "    comm.on_msg(handler);\n"
                          "};\n"
                          "comm_manager.register_target('rate_limiter', rate_limiter);\n");
}

setNotebookOptions();
</script>
'''))


In [11]:
import requests
import pandas as pd

# API URL
api_url = "https://api.binance.com/api/v3/ticker/24hr"

# Define the parameters for the request (optional)
params = {
    # Add any parameters you need here
}

# Send a GET request to the API with optional parameters
response = requests.get(api_url, params=params)
print(response.content)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    try:
        # Attempt to parse the JSON response
        data = response.json()

        # Convert the data to a DataFrame
        df = pd.DataFrame(data)

        # Print available columns to the user
        available_columns = df.columns.tolist()
        print("Available columns:", available_columns)

        # Ask the user for column selection
        user_input = input("Enter columns to keep (comma-separated): ")
        selected_columns = [col.strip() for col in user_input.split(",")]

        # Validate user input and select columns
        columns_to_keep = [col for col in selected_columns if col in available_columns]
        if columns_to_keep:
            df = df[columns_to_keep]

            # Save the DataFrame to a CSV file
            df.to_csv("binance_data.csv", index=False)

            print("CSV file 'binance_data.csv' created successfully.")
        else:
            print("No valid columns selected.")
    except ValueError as e:
        print(f"Error: Unable to parse JSON response. {e}")
else:
    print(f"Error: Unable to fetch data from the API. Status code: {response.status_code}")


b'{\n  "code": 0,\n  "msg": "Service unavailable from a restricted location according to \'b. Eligibility\' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."\n}'
Error: Unable to fetch data from the API. Status code: 451
